Auto tag your Sirv images for easier management with AI.


Run the cells below one by one

# 1. Install Azure SDK



In [2]:
!pip install azure-cognitiveservices-vision-computervision
# Download the demo images file (VM filesystem isn't persistent)
!wget -q https://sirv.sirv.com/website/screenshots/auto-alt-tags/images.txt

     |████████████████████████████████| 85 kB 3.4 MB/s 
     |████████████████████████████████| 41 kB 790 kB/s 


#2. Add API keys

### 2.1 Get Azure key
Follow the instructions <a href="https://sirv.com/help/articles/auto-alt-tags-ai#cv">here</a>
### 2.2 Get Sirv keys
[Follow these instructions](https://sirv.com/help/articles/auto-alt-tags-ai#get-sirv-rest-api-keys)

In [6]:
#@title 2.3 Add your API keys here and then run this cell
#@markdown Azure key

subscription_key = ''  #@param {type: "string"}
#@markdown Sirv keys
sirv_id = '' #@param {type: "string"}
sirv_secret = '' #@param {type: "string"}


# Add your images 
Add images (one image per line) to /content/images.txt

Learn how to get them here - https://sirv.com/help/articles/auto-alt-tags-ai#adding-image-alt-tags-in-bulk

# Run the code

In [22]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials
import json
import requests
import time
from urllib.parse import urlparse

#Create Azure client
endpoint = "https://we.cognitiveservices.azure.com/"
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

# Get Sirv auth token. API reference - https://apidocs.sirv.com/#connect-to-sirv-api
def getToken():
  payload = {
    'clientId': sirv_id,
    'clientSecret': sirv_secret
  }
  headers = {'content-type': 'application/json', "Origin": "*"}
  response = requests.request('POST', 'https://api.sirv.com/v2/token', data=json.dumps(payload), headers=headers)
  token = response.json()['token']
  return token

#We're opening the csv file and looping through each image in the file
with open('images.txt') as f:
  for image in f:
    getToken()
    try:
      urlp = urlparse(image)
      filename = urlp.path
      params = {"filename": filename.replace('\n', '')}
      tagsurl = 'https://api.sirv.com/v2/files/meta/tags'
      # Get the tags
        # Call API with remote image
      tags_result_remote = computervision_client.tag_image(image)

      # Print results with confidence score
      print("Tags in the remote image: ")
      if (len(tags_result_remote.tags) == 0):
          print("No tags detected.")
      else:
          tags = []
          for tag in tags_result_remote.tags:
              print("'{}' with confidence {:.2f}%".format(tag.name, tag.confidence * 100))
              if (tag.confidence*100 > 60):
                  tags.append(tag.name)
          tags = str(tags)
          tags = tags.replace('\'','\"')
          payload='{"tags":'+ str(tags) +'}'
          headers = {
            'content-type': "application/json",
            'authorization': 'Bearer %s' % token
          }
          response = requests.request('POST', tagsurl, data=payload.encode('utf-8'), headers=headers, params=params)
          print(response)
          time.sleep(4)
    except Exception as ex:
        print(ex)
        continue

Tags in the remote image: 
'sky' with confidence 98.95%
'cloud' with confidence 94.55%
'outdoor' with confidence 93.79%
'flag' with confidence 84.28%
'mountain' with confidence 63.03%
<Response [200]>
Tags in the remote image: 
'text' with confidence 99.75%
'human face' with confidence 98.90%
'suit' with confidence 97.30%
'person' with confidence 96.58%
'tie' with confidence 94.62%
'screenshot' with confidence 93.90%
'man' with confidence 89.09%
'clothing' with confidence 88.37%
'forehead' with confidence 87.85%
'white-collar worker' with confidence 84.44%
<Response [200]>
Tags in the remote image: 
'graphic design' with confidence 95.67%
'graphics' with confidence 94.69%
'poster' with confidence 89.85%
'clipart' with confidence 86.00%
'design' with confidence 54.99%
'illustration' with confidence 44.52%
<Response [200]>
Tags in the remote image: 
'person' with confidence 98.61%
'man' with confidence 96.09%
'clothing' with confidence 96.07%
'outdoor' with confidence 95.64%
'demonstrati